<h1>Churn-Detection Using ANN</h1>

<h2>Load Libraries</h2>

In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

<h2>Importing Telcom Churn Dataset</h2>

In [6]:
df=pd.read_csv('Churn-dataset.csv')
df.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
6082,4523-WXCEF,Female,0,Yes,No,59,Yes,Yes,Fiber optic,No,...,No,No,Yes,Yes,Month-to-month,No,Electronic check,101.10,6039.9,Yes
4410,0848-ZGQIJ,Female,0,Yes,No,16,Yes,No,Fiber optic,No,...,No,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),94.65,1461.15,No
6986,3976-BWUCK,Female,0,Yes,No,30,Yes,Yes,Fiber optic,No,...,No,Yes,Yes,No,Month-to-month,Yes,Credit card (automatic),94.10,2804.45,Yes
4911,8565-WUXZU,Male,1,Yes,No,72,Yes,No,Fiber optic,Yes,...,No,Yes,No,No,Two year,Yes,Credit card (automatic),84.80,6141.65,No
1564,2660-EMUBI,Male,1,No,No,10,Yes,Yes,Fiber optic,No,...,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,95.35,898.35,Yes


<h2>Data Preprocessing</h2>

In dataset we can see there are many unwanted things.We'll remove them and clean our data

In [7]:
df.drop('customerID',axis='columns',inplace=True)
df = df.dropna(how='all') # remove samples with all missing values
df = df[~df.duplicated()] # remove duplicates
df.sample(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
5242,Male,0,No,No,20,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,No,Month-to-month,Yes,Electronic check,88.70,1761.45,Yes
6133,Male,0,No,No,2,No,No phone service,DSL,No,No,No,No,No,No,Month-to-month,Yes,Credit card (automatic),24.90,49.7,No
4142,Female,0,No,No,67,Yes,No,DSL,Yes,Yes,No,Yes,No,No,One year,No,Credit card (automatic),60.50,3870,No
203,Male,0,No,Yes,13,Yes,No,Fiber optic,No,No,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),78.30,1033.95,No
179,Male,0,Yes,No,61,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Credit card (automatic),20.55,1252,No


In [8]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

We can see that 'TotalCharges' is of type object,So lets change it to float

In [9]:
#changing TotalCharges from object to float
total_charges_filter = df.TotalCharges == " "
df = df[~total_charges_filter]
df.TotalCharges = pd.to_numeric(df.TotalCharges)
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

<h3>Splitting Dataset into inputs(X) and targets(Y) </h3>

In [10]:
X=df.iloc[:,:19]
Y=df.iloc[:,19:]
X.head()


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65


In [11]:
Y.head()

,Churn
0,No
1,No
2,Yes
3,No
4,Yes


<h3>Converting all values to integer or float</h3>

In [12]:
Y['Churn'].replace({'Yes': 1,'No': 0},inplace=True)
X.replace('No internet service','No',inplace=True)
X.replace('No phone service','No',inplace=True)
X['gender'].replace({'Female':1,'Male':0},inplace=True)

In [13]:
yes_no_columns = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
                  'DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling',]
for col in yes_no_columns:
    X[col].replace({'Yes': 1,'No': 0},inplace=True)

<h3>Converting Categorical Data</h3>

In [14]:
X = pd.get_dummies(data=X, columns=['InternetService','Contract','PaymentMethod'])
X.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_DSL,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,0,1,0,1,0,0,0,1,0,...,1,0,0,1,0,0,0,0,1,0
1,0,0,0,0,34,1,0,1,0,1,...,1,0,0,0,1,0,0,0,0,1
2,0,0,0,0,2,1,0,1,1,0,...,1,0,0,1,0,0,0,0,0,1
3,0,0,0,0,45,0,0,1,0,1,...,1,0,0,0,1,0,1,0,0,0
4,1,0,0,0,2,1,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0


<strong>Removing each column from data to avoid Dummy Variable Trap</strong>

In [15]:
X.drop('InternetService_No',axis='columns',inplace=True)
X.drop('Contract_Month-to-month',axis='columns',inplace=True)
X.drop('PaymentMethod_Bank transfer (automatic)',axis='columns',inplace=True)


In [16]:
X.dtypes

gender                                     int64
SeniorCitizen                              int64
Partner                                    int64
Dependents                                 int64
tenure                                     int64
PhoneService                               int64
MultipleLines                              int64
OnlineSecurity                             int64
OnlineBackup                               int64
DeviceProtection                           int64
TechSupport                                int64
StreamingTV                                int64
StreamingMovies                            int64
PaperlessBilling                           int64
MonthlyCharges                           float64
TotalCharges                             float64
InternetService_DSL                        uint8
InternetService_Fiber optic                uint8
Contract_One year                          uint8
Contract_Two year                          uint8
PaymentMethod_Credit

<h3>Splitting dataset into Test and Train </h3>

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [18]:
X_train.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,PaperlessBilling,MonthlyCharges,TotalCharges,InternetService_DSL,InternetService_Fiber optic,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
1251,1,0,0,0,1,1,0,0,1,0,...,0,75.80,75.80,0,1,0,0,0,1,0
676,1,0,0,0,48,1,1,0,0,0,...,0,24.55,1203.95,0,0,1,0,0,0,0
381,0,0,0,0,1,1,0,0,1,0,...,0,50.65,50.65,1,0,0,0,0,0,1
2313,0,0,1,1,43,1,1,0,0,1,...,1,104.30,4451.85,0,1,1,0,1,0,0
6901,1,0,0,0,30,1,1,0,1,1,...,1,68.95,2038.70,1,0,1,0,1,0,0


In [19]:
Y_train.head()

,Churn
1251,1
676,0
381,1
2313,0
6901,0


<h2>Training ANN Model for Churn-Detection</h2>

In [20]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [21]:
model = Sequential()
model.add(Dense(23,  kernel_initializer='uniform', input_shape=(23,), activation='relu'))
model.add(Dense(23,  kernel_initializer='uniform', input_shape=(23,), activation='relu'))
model.add(Dense(1,  kernel_initializer='uniform', input_shape=(23,), activation='sigmoid'))


In [22]:
model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=10 ,epochs=100)


Epoch 1/100


InternalError:  Blas GEMM launch failed : a.shape=(10, 23), b.shape=(23, 23), m=10, n=23, k=23
	 [[node sequential_1/dense_3/MatMul (defined at <ipython-input-22-9ac876f01a29>:4) ]] [Op:__inference_train_function_814]

Function call stack:
train_function


<strong>Saving model in model.h5 file</strong>

In [ ]:
model.save("model.h5")

In [ ]:
#model with acuuracy of 80.8% if needed
# model=keras.models.load_model("final_model.h5")
# print(m)

<h3>Evaluating our ANN model</h3>

In [ ]:
Y_pred=model.predict(X_test)
Y_pred=(Y_pred>0.5)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
cm=confusion_matrix(Y_test,Y_pred)
print(classification_report(Y_test,Y_pred))

In [ ]:
import seaborn as sn
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
#accuracy=(TP+TN)/total
(901+216)/1407